Ferramentas de NLP usando o dataset de tweets

Modelo SVM

70/30 Treinamento


In [ ]:
from google.colab import drive
from collections import Counter
import pandas as pd
import nltk
from nltk.tokenize.casual import casual_tokenize
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from numpy import array
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn import preprocessing
from nltk.util import ngrams

Dando permissão para o colab poder acessar os arquivos do meu drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/My Drive/NLP/DataSet da Primeira Competição/train.csv')

# df.head()

X = df.Text
y = df.Classificacao

# y = pd.get_dummies(y)

Transformar palavras em números.

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Pre processamento Junto com Bag of Words

Pré Processamento 1 = remoção de stopwords, remoção de link, pontuação com bag of words TF.

Pré Processamento 2 = remoção de stopwords, pontuação com bag of words TF.

Pré Processamento 3 = pontuação com bag of words TF.

Pré Processamento 4 = remoção de stopwords com bag of words TF.

In [ ]:
stop_words_ptbr = nltk.corpus.stopwords.words('portuguese')
stop_words_en = nltk.corpus.stopwords.words('english')
stop_words_spn = nltk.corpus.stopwords.words('spanish')

stop_words = stop_words_ptbr + stop_words_en + stop_words_spn

bags_of_words = []

vocabulary = []

for x in X:
  aux = ""
  x = re.sub(r'http\S+', '', x)
  tokens = casual_tokenize(x, reduce_len=True, strip_handles=True)
  puncs = set((',', '.', '--','-','!','?',':', ';', '``', "''", '(', ')', '[', ']','\u2026','"',"/"))
  tokens = (x.lower() for x in tokens if x not in puncs)
  tokens = Counter(tokens)
  tokens = [x for x in tokens if x not in stop_words]
  bags_of_words.append(Counter(tokens))

  vocabulary += [" ".join([x for x in tokens])]

df_bows = pd.DataFrame.from_records(bags_of_words)
df_bows = df_bows.fillna(0).astype(int)

In [ ]:
vectorizer = TfidfVectorizer(min_df=1)
tf_idf = vectorizer.fit_transform(vocabulary)

In [ ]:
tf_idf.shape

(6559, 5560)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_bows,y, test_size = 0.3, random_state = 109)

Inverse do OneHotEncode

In [ ]:
# pd.get_dummies(y).idxmax(1)

In [ ]:
y_train

1386    Positivo
1579    Positivo
5127    Negativo
6166    Negativo
4234    Negativo
          ...   
2635      Neutro
2447    Positivo
141     Negativo
3317    Negativo
3334    Positivo
Name: Classificacao, Length: 4591, dtype: object

Regressão logistica não precisa transformar as palavras em números, outros modelos também como knn e svm.

In [ ]:
# parameters = {'kernel':('linear', 'rbf', 'poly'), 'C':[1, 10], 'tol':[0.001,0.1]}

# svc = svm.SVC()
# clf = GridSearchCV(svc, parameters)

parameters = {'n_neighbors':[1,100],'metric':('euclidean','manhattan','minkowski')}

# svc = KNeighborsClassifier()
# svc = KNeighborsClassifier(n_neighbors= = 1, metric='euclidean')
# clf = GridSearchCV(svc, parameters)

# parameters = {'solver':('newton-cg','sag','saga','lbfgs'),'tol':[0.0001,0.1],'C':[1,10]}

# svc = LogisticRegression()
# clf = GridSearchCV(svc, parameters)

# clf = svm.SVC(decision_function_shape="ovr", kernel= "linear")
# clf = KNeighborsClassifier(n_neighbors=1,metric='manhattan') #0.9390
# clf = MultinomialNB()
# clf = LogisticRegression(multi_class='multinomial', solver='lbfgs',tol=0.1) #0.9603

clf.fit(X_train, y_train)

SyntaxError: ignored

In [ ]:
print(clf.best_estimator_)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')


In [ ]:
-grid_predictions = clf.predict(X_test)-
print(confusion_matrix(y_test,grid_predictions))
print(classification_report(y_test,grid_predictions))

[[522  42   1]
 [ 10 606   8]
 [  0  26 753]]
              precision    recall  f1-score   support

    Negativo       0.98      0.92      0.95       565
      Neutro       0.90      0.97      0.93       624
    Positivo       0.99      0.97      0.98       779

    accuracy                           0.96      1968
   macro avg       0.96      0.95      0.95      1968
weighted avg       0.96      0.96      0.96      1968



(1968,)

In [ ]:
predict = clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score as acs

print(acs(predict, y_test))

0.9542682926829268
